In [1]:
!pip install git+https://github.com/AAMIASoftwares-research/HCATNetwork.git@google-colab
!pip install git+https://github.com/AAMIASoftwares-research/DatasetUtilities.git@google-colab

  Cloning https://github.com/AAMIASoftwares-research/HCATNetwork.git (to revision google-colab) to c:\users\zaira\appdata\local\temp\pip-req-build-wp0msaj2
  Resolved https://github.com/AAMIASoftwares-research/HCATNetwork.git to commit e94413cce5e4fad1a00a771112bf7e66e3501d4d
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/AAMIASoftwares-research/HCATNetwork.git 'C:\Users\zaira\AppData\Local\Temp\pip-req-build-wp0msaj2'
  Running command git checkout -b google-colab --track origin/google-colab
  branch 'google-colab' set up to track 'origin/google-colab'.
  Switched to a new branch 'google-colab'


  Cloning https://github.com/AAMIASoftwares-research/DatasetUtilities.git (to revision google-colab) to c:\users\zaira\appdata\local\temp\pip-req-build-b87b9a0o
  Resolved https://github.com/AAMIASoftwares-research/DatasetUtilities.git to commit abbffcb17dc017057ea1db6c361aecba70cf76f5
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Cloning https://github.com/AAMIASoftwares-research/HCATNetwork.git (to revision google-colab) to c:\users\zaira\appdata\local\temp\pip-install-3wetdz7p\hcatnetwork_4737e430312747c8862df3ff9232c7ed
  Resolved https://github.com/AAMIASoftwares-research/HCATNetwork.git to commi

  Running command git clone --filter=blob:none --quiet https://github.com/AAMIASoftwares-research/DatasetUtilities.git 'C:\Users\zaira\AppData\Local\Temp\pip-req-build-b87b9a0o'
  Running command git checkout -b google-colab --track origin/google-colab
  branch 'google-colab' set up to track 'origin/google-colab'.
  Switched to a new branch 'google-colab'
  Running command git clone --filter=blob:none --quiet https://github.com/AAMIASoftwares-research/HCATNetwork.git 'C:\Users\zaira\AppData\Local\Temp\pip-install-3wetdz7p\hcatnetwork_4737e430312747c8862df3ff9232c7ed'
  Running command git checkout -b google-colab --track origin/google-colab
  branch 'google-colab' set up to track 'origin/google-colab'.
  Switched to a new branch 'google-colab'


In [2]:
import hcatnetwork
import HearticDatasetManager
import networkx

In [3]:
import os
import numpy
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [4]:
# import graph annotations
from HearticDatasetManager.cat08.dataset import DATASET_CAT08_GRAPHS_RESAMPLED_05MM
from HearticDatasetManager.asoca.dataset import DATASET_ASOCA_GRAPHS_RESAMPLED_05MM_DICT
# import images
from HearticDatasetManager.asoca import AsocaImageCT
from HearticDatasetManager.asoca.dataset import DATASET_ASOCA_IMAGES_DICT
from HearticDatasetManager.cat08 import Cat08ImageCT
from HearticDatasetManager.cat08.dataset import DATASET_CAT08_IMAGES

The goal of this notebook is to test the labeling of the volume patches

In [5]:
import pickle 
# import data C:/Users/zaira/Documents/uni/HIGH_PERFORMANCE_COMPUTING/Neuroengineering/project/heart_data/MY_ASOCA/asoca_img_0_lin'
folder_img = 'C:/Users/zaira/Documents/uni/HIGH_PERFORMANCE_COMPUTING/Neuroengineering/project/MY_ASOCA/asoca_img_0_lin'
folder_coord = 'C:/Users/zaira/Documents/uni/HIGH_PERFORMANCE_COMPUTING/Neuroengineering/project/MY_ASOCA/coord_asoca_img_0_lin'

with open(folder_img, 'rb') as file:
    image = pickle.load(file)
with open(folder_coord, 'rb') as file:
    coords = pickle.load(file)

to have coordinates of the ostium I need to import the graph

In [8]:
# now perform graph transformation to RAS coordinates 
# load graph
asoca_dataset_folder = "C:/Users/zaira/Documents/uni/HIGH_PERFORMANCE_COMPUTING/Neuroengineering/project/heart_data/ASOCA/"
asoca_graph_file = os.path.join(
    asoca_dataset_folder,
    DATASET_ASOCA_GRAPHS_RESAMPLED_05MM_DICT["Normal"][0]
)

graph_asoca = hcatnetwork.io.load_graph(
    file_path=asoca_graph_file,
    output_type=hcatnetwork.graph.SimpleCenterlineGraph
)

pts = []
for i, v in enumerate(graph_asoca.nodes.values()):
    pts.append([v["x"],v["y"],v["z"],v['r'],v['topology']])

pts = numpy.array(pts)
print(pts)


[[230.02615356445312 243.5576934814453 -89.78585052490234
  1.8917913147124796 <ArteryNodeTopology.INTERSECTION: 3>]
 [230.47400135567534 243.56299092373087 -90.02835183160373
  1.8432867781293303 <ArteryNodeTopology.SEGMENT: 2>]
 [230.8547878343426 243.62986855509962 -90.36635773709583
  1.717448218195238 <ArteryNodeTopology.SEGMENT: 2>]
 ...
 [273.9672007304628 222.95288158033858 -108.53065590655335
  0.4223645926732999 <ArteryNodeTopology.SEGMENT: 2>]
 [274.0401234578792 222.52641925009334 -108.79841007815978
  0.3571271076759852 <ArteryNodeTopology.SEGMENT: 2>]
 [274.2112121582031 222.06509399414062 -108.79074096679688
  0.19999647341230686 <ArteryNodeTopology.ENDPOINT: 4>]]


In [12]:
# need to remove the radius and topology to apply transform
pts_no_top = np.array([subarray[:-2] for subarray in pts])

In [13]:
# apply transform to graph/image 
from HearticDatasetManager.affine import apply_affine_3d

transform = image.affine_centerlines2ras
pts_no_top = apply_affine_3d(transform, pts_no_top.T).T

In [14]:
# now add again the radius and topology information
pts_no_top = np.concatenate((pts_no_top, pts[:, -2][:, np.newaxis],pts[:, -1][:, np.newaxis]), axis=1)
pts_no_top.shape

(1287, 5)

In [15]:
# retrive the indices of the ostium coordinates
ls = []
for i,entry in enumerate(pts_no_top):
    if (entry[-1].value == 1):
        ls.append(i)
        
pts_no_top[ls]

array([[13.183456420898438, -1.388641357421875, -93.45762634277344,
        2.180273815753563, <ArteryNodeTopology.OSTIUM: 1>],
       [20.466461181640625, 28.725494384765625, -104.70037841796875,
        1.4173272872714284, <ArteryNodeTopology.OSTIUM: 1>]], dtype=object)

In [31]:
# save x, y, z coords and radius
info_ostium_1 = pts_no_top[ls][0][:4]
info_ostium_2 = pts_no_top[ls][1][:4]
print("ostium1:",info_ostium_1,"\n ostium2: ", info_ostium_2)

ostium1: [13.183456420898438 -1.388641357421875 -93.45762634277344
 2.180273815753563] 
 ostium2:  [20.466461181640625 28.725494384765625 -104.70037841796875
 1.4173272872714284]


Now split the original volume into smaller ones

In [36]:
#function to see the if the percentage of black pixels in the image is above a certain threshold (30)
# it returns True if the percentage of black voxels is greater than or equal to the specified threshold.

def check_black_percentage(volume, threshold=30):
    total_voxels = np.prod(volume.shape)
    black_voxels = np.count_nonzero(volume == 0)
    percentage_black = (black_voxels / total_voxels) * 100
    print("this volume has a percentage of black voxels equal to: ", percentage_black, "%")
    return percentage_black >= threshold

In [17]:
# split original volume
def split_3d_volume(volume, coordinates, patch_size, overlap, threshold=0.3):
    patches = []
    for x in range(0, volume.shape[0] - patch_size[0] + 1, overlap[0]):
        for y in range(0, volume.shape[1] - patch_size[1] + 1, overlap[1]):
            for z in range(0, volume.shape[2] - patch_size[2] + 1, overlap[2]):
                patch = volume[x:x+patch_size[0], y:y+patch_size[1], z:z+patch_size[2]]                
                coord = coordinates[x:x+patch_size[0], y:y+patch_size[1], z:z+patch_size[2]]
                patches.append({'value': patch, 'coord': coord})
                

    return patches

volume = image.data
patch_size = (32, 32, 32)
overlap = (32, 32, 32)
patches = split_3d_volume(volume, coords, patch_size, overlap)

In [37]:
for xx in range(patch_size[0]):
    for yy in range(patch_size[1]):
        for zz in range(patch_size[2]):
            check_black_percentage(volume[xx,yy,zz])
print(check_black_percentage)

this volume has a percentage of black voxels equal to:  0.0 %
this volume has a percentage of black voxels equal to:  0.0 %
this volume has a percentage of black voxels equal to:  0.0 %
this volume has a percentage of black voxels equal to:  0.0 %
this volume has a percentage of black voxels equal to:  0.0 %
this volume has a percentage of black voxels equal to:  0.0 %
this volume has a percentage of black voxels equal to:  0.0 %
this volume has a percentage of black voxels equal to:  0.0 %
this volume has a percentage of black voxels equal to:  0.0 %
this volume has a percentage of black voxels equal to:  0.0 %
this volume has a percentage of black voxels equal to:  0.0 %
this volume has a percentage of black voxels equal to:  0.0 %
this volume has a percentage of black voxels equal to:  0.0 %
this volume has a percentage of black voxels equal to:  0.0 %
this volume has a percentage of black voxels equal to:  0.0 %
this volume has a percentage of black voxels equal to:  0.0 %
this vol

In [18]:
# check number of patches
len(patches)


1183

In [19]:
# check dimanesionality of volume patch
patches[0]['value'].shape

(32, 32, 32)

In [20]:
# check dimanesionality of coordinates of volume patch
patches[110]['coord'].shape

(32, 32, 32, 4)

Now label the individual patches

In [21]:
def label_patches_with_sphere(patches, center, radius, threshold=0.1):
    
    voxel_count = np.prod(patches[0]['value'].shape)
    
    labeled_patches = []

    for patch in patches:
        coordinates = patch['coord']

        distances_from_center = []

        for i in range(coordinates.shape[0]):
            for j in range(coordinates.shape[1]):
                for k in range(coordinates.shape[2]):
                    distances_from_center.append(np.sqrt(np.sum(np.square(coordinates[i, j, k][:3] - center))))
        
        distances_from_center = np.array(distances_from_center)
        #print(distances_from_center.shape) check that shape is equal to voxel_count
        inside_sphere_count = np.sum(distances_from_center <= radius)
        inside_sphere_percentage = inside_sphere_count / voxel_count

        label = 1 if inside_sphere_percentage > threshold else 0
        labeled_patches.append({'value': patch['value'], 'coord': patch['coord'], 'label': label})

    return labeled_patches


center = info_ostium_1[0:3]
radius = info_ostium_1[-1]
labeled_patches = label_patches_with_sphere(patches, center, radius)


In [22]:
count_label_1 = 0
count_label_0 = 0
for labelled in labeled_patches:
    if(labelled['label']==1):
        count_label_1 += 1
    else:
        count_label_0 += 1
        
print(count_label_1)
print(count_label_0)

0
1183


In [ ]:
# see if each volume has more than a given threshold of black voxels
def split_3d_volume(volume, coordinates, patch_size, overlap):
    patches = []
    for x in range(0, volume.shape[0] - patch_size[0] + 1, overlap[0]):
        for y in range(0, volume.shape[1] - patch_size[1] + 1, overlap[1]):
            for z in range(0, volume.shape[2] - patch_size[2] + 1, overlap[2]):
                patch = volume[x:x+patch_size[0], y:y+patch_size[1], z:z+patch_size[2]]
                coord = coordinates[x:x+patch_size[0], y:y+patch_size[1], z:z+patch_size[2]]
                patches.append({'value': patch, 'coord': coord})
    return patches

volume = image.data
patch_size = (32, 32, 32)
overlap = (32, 32, 32)
patches = split_3d_volume(volume, coords, patch_size, overlap)

In [ ]:
# # split original volume
# def split_3d_volume(volume, coordinates, patch_size, overlap, threshold=0.3):
#     patches = []
#     for x in range(0, volume.shape[0] - patch_size[0] + 1, overlap[0]):
#         for y in range(0, volume.shape[1] - patch_size[1] + 1, overlap[1]):
#             for z in range(0, volume.shape[2] - patch_size[2] + 1, overlap[2]):
#                 patch = volume[x:x+patch_size[0], y:y+patch_size[1], z:z+patch_size[2]]
                
#                 coord = coordinates[x:x+patch_size[0], y:y+patch_size[1], z:z+patch_size[2]]
#                 patches.append({'value': patch, 'coord': coord})

#     return patches

# volume = image.data
# patch_size = (32, 32, 32)
# overlap = (32, 32, 32)
# patches = split_3d_volume(volume, coords, patch_size, overlap)